# Leetcode Premium Problems

In [2]:
from typing import List
from typing import Optional
import pandas as pd
import numpy as np

## 1757. Recyclable and Low Fat Products [E]

Table: Products
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| low_fats    | enum    |
| recyclable  | enum    |
+-------------+---------+
product_id is the primary key (column with unique values) for this table.
low_fats is an ENUM (category) of type ('Y', 'N') where 'Y' means this product is low fat and 'N' means it is not.
recyclable is an ENUM (category) of types ('Y', 'N') where 'Y' means this product is recyclable and 'N' means it is not.
```

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

The result format is in the following example.



In [6]:
# Beats 33.89%
def find_products(products: pd.DataFrame) -> pd.DataFrame:
    return products.loc[ (products['low_fats']=='Y') & (products['recyclable']=='Y'), ['product_id'] ]

In [3]:
data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype({'product_id':'int64', 'low_fats':'category', 'recyclable':'category'})

In [9]:
find_products(products)

,product_id


## 2877. Create a DataFrame from List [E]

Write a solution to create a DataFrame from a 2D list called `student_data`. This 2D list contains the IDs and ages of some students.

The DataFrame should have two columns, `student_id` and `age`, and be in the same order as the original 2D list.

The result format is in the following example.

In [12]:
# Beats 5.06%
def createDataframe(student_data: List[List[int]]) -> pd.DataFrame:
    return pd.DataFrame(student_data, columns=['student_id', 'age'])

In [10]:
student_data = [
  [1, 15],
  [2, 11],
  [3, 11],
  [4, 20]
]

In [13]:
createDataframe(student_data)

,student_id,age
0,1,15
1,2,11
2,3,11
3,4,20


## 1661. Average Time of Process per Machine [E]

Table: Activity
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| machine_id     | int     |
| process_id     | int     |
| activity_type  | enum    |
| timestamp      | float   |
+----------------+---------+
The table shows the user activities for a factory website.
(machine_id, process_id, activity_type) is the primary key (combination of columns with unique values) of this table.
machine_id is the ID of a machine.
process_id is the ID of a process running on the machine with ID machine_id.
activity_type is an ENUM (category) of type ('start', 'end').
timestamp is a float representing the current time in seconds.
'start' means the machine starts the process at the given timestamp and 'end' means the machine ends the process at the given timestamp.
The 'start' timestamp will always be before the 'end' timestamp for every (machine_id, process_id) pair.
It is guaranteed that each (machine_id, process_id) pair has a 'start' and 'end' timestamp.
```

There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

The time to complete a process is the `'end' timestamp` minus the `'start' timestamp`. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the `machine_id` along with the average time as `processing_time`, which should be rounded to 3 decimal places.

Return the result table in any order.

The result format is in the following example.

In [73]:
# Beats 33.89%
def get_average_time(activity: pd.DataFrame) -> pd.DataFrame:
    if activity.shape[0] < 1:
        return pd.DataFrame(columns=['machine_id','processing_time'])
    df = activity.pivot(index=['machine_id','process_id'], columns=['activity_type'], values=['timestamp']).reset_index().droplevel(level=1, axis=1)
    df.columns = ['machine_id','process_id','end','start']
    df['ttime'] = df['end'] - df['start']
    return df.groupby('machine_id', as_index=False).agg(processing_time=('ttime', lambda x: round(np.mean(x) + 1e-9, 3)))

In [74]:
data = [[0, 0, 'start', 0.712], [0, 0, 'end', 1.52], [0, 1, 'start', 3.14], [0, 1, 'end', 4.12], [1, 0, 'start', 0.55], [1, 0, 'end', 1.55], [1, 1, 'start', 0.43], [1, 1, 'end', 1.42], [2, 0, 'start', 4.1], [2, 0, 'end', 4.512], [2, 1, 'start', 2.5], [2, 1, 'end', 5]]
activity = pd.DataFrame(data, columns=['machine_id', 'process_id', 'activity_type', 'timestamp']).astype({'machine_id':'Int64', 'process_id':'Int64', 'activity_type':'object', 'timestamp':'Float64'})

In [75]:
get_average_time(activity)

,machine_id,processing_time
0,0,0.894
1,1,0.995
2,2,1.456


## 1731. The Number of Employees Which Report to Each Employee [E]

Table: Employees
```
+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| reports_to  | int      |
| age         | int      |
+--------`-----+----------+
employee_id is the column with unique values for this table.
This table contains information about the employees and the id of the manager they report to. Some employees do not report to anyone (reports_to is null). 
```

For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

In [103]:
# Beats 5.71%
def count_employees(employees: pd.DataFrame) -> pd.DataFrame:
    managers = employees.copy()
    managers = managers[['employee_id','name','age']]
    managers.columns = ['reports_to','name_m','age_m']
    df = pd.merge(
        employees, managers, how='inner', on='reports_to'
    ).groupby(
        ['reports_to','name_m'], as_index=False
    ).agg(
        reports_count=('employee_id','count'), average_age=('age',lambda x: round(np.mean(x)+1e-9,0),)
    )
    df.columns = ['employee_id','name','reports_count','average_age']
    return df

In [98]:
data = [[9, 'Hercy', None, 43], [6, 'Alice', 9, 41], [4, 'Bob', 9, 36], [2, 'Winston', None, 37]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'reports_to', 'age']).astype({'employee_id':'Int64', 'name':'object', 'reports_to':'Int64', 'age':'Int64'})

In [106]:
count_employees(employees)

,employee_id,name,reports_count,average_age


## 1978. Employees Whose Manager Left the Company [E]

Table: Employees
```
+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| manager_id  | int      |
| salary      | int      |
+-------------+----------+
In SQL, employee_id is the primary key for this table.
This table contains information about the employees, their salary, and the ID of their manager. Some employees do not have a manager (manager_id is null). 
```

Find the IDs of the employees whose salary is strictly less than $30000 and whose manager left the company. When a manager leaves the company, their information is deleted from the Employees table, but the reports still have their manager_id set to the manager that left.

Return the result table ordered by employee_id.

The result format is in the following example.

In [120]:
# Beats 62.96%
def find_employees(employees: pd.DataFrame) -> pd.DataFrame:
    return employees.loc[ (~employees['manager_id'].isin(employees['employee_id'])) & (~employees['manager_id'].isna()) & (employees['salary']<30000), ['employee_id']].sort_values('employee_id')

In [118]:
data = [[3, 'Mila', 9, 60301], [12, 'Antonella', None, 31000], [13, 'Emery', None, 67084], [1, 'Kalel', 11, 21241], [9, 'Mikaela', None, 50937], [11, 'Joziah', 6, 28485]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'manager_id', 'salary']).astype({'employee_id':'Int64', 'name':'object', 'manager_id':'Int64', 'salary':'Int64'})

In [119]:
find_employees(employees)

,employee_id
5,11


## 1384. Total Sales Amount by Year [H]

Table: Product
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| product_name  | varchar |
+---------------+---------+
product_id is the primary key (column with unique values) for this table.
product_name is the name of the product.
```

Table: Sales
```
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| product_id          | int     |
| period_start        | date    |
| period_end          | date    |
| average_daily_sales | int     |
+---------------------+---------+
product_id is the primary key (column with unique values) for this table. 
period_start and period_end indicate the start and end date for the sales period, and both dates are inclusive.
The average_daily_sales column holds the average daily sales amount of the items for the period.
The dates of the sales years are between 2018 to 2020.
```

Write a solution to report the total sales amount of each item for each year, with corresponding product_name, product_id, report_year, and total_amount.

Return the result table ordered by product_id and report_year.

The result format is in the following example.

In [220]:
# Beats 42.86%
def total_sales(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    sales['start2018'] = (pd.to_datetime('2019-01-01') - sales['period_start']).dt.days # if pos, then starts in 2018
    sales['end2018'] = (pd.to_datetime('2019-01-01') - sales['period_end']).dt.days - 1 # if pos, then ends in 2018, subtract one to account for final day
    sales.loc[ sales['start2018']<0, ['start2018'] ] = 0
    sales.loc[ sales['end2018']<0, ['end2018'] ] = 0
    sales['days2018'] = (sales['start2018'] - sales['end2018'])
    sales['total_amount2018'] = sales['days2018'] * sales['average_daily_sales']

    sales['start2019'] = (pd.to_datetime('2020-01-01') - sales['period_start']).dt.days - sales['days2018'] # remove days from prev year
    sales['end2019'] = (pd.to_datetime('2020-01-01') - sales['period_end']).dt.days - 1
    sales.loc[ sales['start2019']<0, ['start2019'] ] = 0
    sales.loc[ sales['end2019']<0, ['end2019'] ] = 0
    sales['days2019'] = (sales['start2019'] - sales['end2019'])
    sales['total_amount2019'] =  sales['days2019'] * sales['average_daily_sales']

    sales['start2020'] = (pd.to_datetime('2021-01-01') - sales['period_start']).dt.days - sales['days2018'] - sales['days2019'] # remove days from prev year(s)
    sales['end2020'] = (pd.to_datetime('2021-01-01') - sales['period_end']).dt.days - 1
    sales.loc[ sales['start2020']<0, ['start2020'] ] = 0
    sales.loc[ sales['end2020']<0, ['end2020'] ] = 0
    sales['days2020'] = (sales['start2020'] - sales['end2020'])
    sales['total_amount2020'] =  sales['days2020'] * sales['average_daily_sales']

    df = pd.merge(sales, product, how='left', on='product_id')[['product_id','product_name','total_amount2018','total_amount2019','total_amount2020']]
    df = pd.wide_to_long(df, stubnames=['total_amount'], i=['product_id','product_name'], j='report_year').reset_index()
    df['report_year'] = df['report_year'].astype(str)
    return df[ df['total_amount']>0 ]

In [221]:
data = [[1, 'LC Phone '], [2, 'LC T-Shirt'], [3, 'LC Keychain']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})
data = [[1, '2019-01-25', '2019-02-28', 100], [2, '2018-12-01', '2020-01-01', 10], [3, '2019-12-01', '2020-01-31', 1]]
sales = pd.DataFrame(data, columns=['product_id', 'period_start', 'period_end', 'average_daily_sales']).astype({'product_id':'Int64', 'period_start':'datetime64[ns]', 'period_end':'datetime64[ns]', 'average_daily_sales':'Int64'})

In [222]:
total_sales(product, sales)

,product_id,product_name,report_year,total_amount
1,1,LC Phone,2019,3500
3,2,LC T-Shirt,2018,310
4,2,LC T-Shirt,2019,3650
5,2,LC T-Shirt,2020,10
7,3,LC Keychain,2019,31
8,3,LC Keychain,2020,31


## 1683. Invalid Tweets [E]

Table: Tweets
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| tweet_id       | int     |
| content        | varchar |
+----------------+---------+
tweet_id is the primary key (column with unique values) for this table.
This table contains all the tweets in a social media app.
```

Write a solution to find the IDs of the invalid tweets. The tweet is invalid if the number of characters used in the content of the tweet is strictly greater than 15.

Return the result table in any order.

The result format is in the following example.

In [229]:
# Beats 75.19%
def invalid_tweets(tweets: pd.DataFrame) -> pd.DataFrame:
    return tweets.loc[ tweets['content'].str.len() > 15, ['tweet_id'] ]

In [231]:
data = [[1, 'Let us Code'], [2, 'More than fifteen chars are here!']]
tweets = pd.DataFrame(data, columns=['tweet_id', 'content']).astype({'tweet_id':'Int64', 'content':'object'})

In [232]:
invalid_tweets(tweets)

,tweet_id
1,2


## 1667. Fix Names in a Table [E]

Table: Users
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| user_id        | int     |
| name           | varchar |
+----------------+---------+
user_id is the primary key (column with unique values) for this table.
This table contains the ID and the name of the user. The name consists of only lowercase and uppercase characters.
```

Write a solution to fix the names so that only the first character is uppercase and the rest are lowercase.

Return the result table ordered by user_id.

The result format is in the following example.



In [249]:
# Beasts 66.47%
def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([users['user_id'], users['name'].str.lower().str.capitalize()], axis=1).sort_values('user_id')

In [247]:
data = [[1, 'aLice'], [2, 'bOB']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})

In [250]:
fix_names(users)

,user_id,name
0,1,Alice
1,2,Bob


## 2878. Get the Size of a DataFrame [E]
```
DataFrame players:
+-------------+--------+
| Column Name | Type   |
+-------------+--------+
| player_id   | int    |
| name        | object |
| age         | int    |
| position    | object |
| ...         | ...    |
+-------------+--------+
```
Write a solution to calculate and display the number of rows and columns of players.

Return the result as an array:

[number of rows, number of columns]

The result format is in the following example.

In [255]:
# Beats 55.84%
def getDataframeSize(players: pd.DataFrame) -> List[int]:
    return list(players.shape)

In [256]:
getDataframeSize(users)

[2, 2]

## 1633. Percentage of Users Attended a Contest [E]

Table: Users
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| user_id     | int     |
| user_name   | varchar |
+-------------+---------+
user_id is the primary key (column with unique values) for this table.
Each row of this table contains the name and the id of a user.
```

Table: Register
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| contest_id  | int     |
| user_id     | int     |
+-------------+---------+
(contest_id, user_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table contains the id of a user and the contest they registered into.
```

Write a solution to find the percentage of the users registered in each contest rounded to two decimals.

Return the result table ordered by percentage in descending order. In case of a tie, order it by contest_id in ascending order.

The result format is in the following example.



In [290]:
# Beats 54.62%
def users_percentage(users: pd.DataFrame, register: pd.DataFrame) -> pd.DataFrame:
    total_users = users.shape[0]
    df = register.groupby('contest_id', as_index=False).agg(count=('user_id','count'))
    df['percentage'] = round(df['count']/total_users*100 + 1e-9, 2)
    return df[['contest_id','percentage']].sort_values(by=['percentage','contest_id'], ascending=[False, True])
    # return df[['contest_id','percentage']].sort_values(['contest_id'], ascending=True).sort_values(['percentage'], ascending=False)

In [285]:
data = [[6, 'Alice'], [2, 'Bob'], [7, 'Alex']]
users = pd.DataFrame(data, columns=['user_id', 'user_name']).astype({'user_id':'Int64', 'user_name':'object'})
data = [[215, 6], [209, 2], [208, 2], [210, 6], [208, 6], [209, 7], [209, 6], [215, 7], [208, 7], [210, 2], [207, 2], [210, 7]]
register = pd.DataFrame(data, columns=['contest_id', 'user_id']).astype({'contest_id':'Int64', 'user_id':'Int64'})

In [291]:
users_percentage(users, register)

,contest_id,percentage
1,208,100.0
2,209,100.0
3,210,100.0
4,215,66.67
0,207,33.33


## 1789. Primary Department for Each Employee [E]

Table: Employee
```
+---------------+---------+
| Column Name   |  Type   |
+---------------+---------+
| employee_id   | int     |
| department_id | int     |
| primary_flag  | varchar |
+---------------+---------+
(employee_id, department_id) is the primary key (combination of columns with unique values) for this table.
employee_id is the id of the employee.
department_id is the id of the department to which the employee belongs.
primary_flag is an ENUM (category) of type ('Y', 'N'). If the flag is 'Y', the department is the primary department for the employee. If the flag is 'N', the department is not the primary.
```

Employees can belong to multiple departments. When the employee joins other departments, they need to decide which department is their primary department. Note that when an employee belongs to only one department, their primary column is 'N'.

Write a solution to report all the employees with their primary department. For employees who belong to one department, report their only department.

Return the result table in any order.

The result format is in the following example.

In [316]:
# Beats 50.23%
def find_primary_department(employee: pd.DataFrame) -> pd.DataFrame:
    employee['duplicated'] = employee['employee_id'].duplicated()

    duplicated = employee.groupby('employee_id', as_index=False).agg(count=('employee_id','count'))

    not_duplicated = duplicated[ duplicated['count']==1 ]

    return pd.concat([employee[ employee['primary_flag'] == 'Y' ], employee[ employee['employee_id'].isin(not_duplicated['employee_id'])]], axis=0)[['employee_id','department_id']]

In [317]:
data = [['1', '1', 'N'], ['2', '1', 'Y'], ['2', '2', 'N'], ['3', '3', 'N'], ['4', '2', 'N'], ['4', '3', 'Y'], ['4', '4', 'N']]
employee = pd.DataFrame(data, columns=['employee_id', 'department_id', 'primary_flag']).astype({'employee_id':'Int64', 'department_id':'Int64', 'primary_flag':'object'})

In [318]:
find_primary_department(employee)

,employee_id,department_id
1,2,1
5,4,3
0,1,1
3,3,3
